In [24]:
import numpy as np

In [25]:
class Markov:
    def __init__(self, categories):
        self.categories = categories
        self.category_map = { categories[i]:i for i in range(len(categories)) }
        self.num_nodes = len(categories)
        self.graph = np.ones((self.num_nodes, self.num_nodes))

    def forward_step(self, start_node, next_node, reward=1):
        u = self.category_map[start_node]
        v = self.category_map[next_node]
        self.graph[u][v]+=reward

    def predict_next(self, node):
        u = self.category_map[node]
        transition_probabilities = self.graph[u] / np.sum(self.graph[u])
        choice = np.random.choice(self.categories, p=transition_probabilities)
        return choice


cat = ["ah", "hello", "clothes", "shoes"]
mv = Markov(cat)


for i in range(100):
    mv.forward_step(np.random.choice(cat), np.random.choice(cat))


print(mv.predict_next("hello"))


ah


In [26]:
class UCB:
    def __init__(self, categories):
        self.num_arms = len(categories)
        self.categories = categories
        self.category_map = { categories[i]:i for i in range(len(categories)) }
        self.num_pulls = np.zeros(self.num_arms)
        self.num_successes = np.zeros(self.num_arms)
        

    def simulate(self, users_choices):
        num_rounds = len(users_choices)
        for i in range(num_rounds):
            ucb_values = self.num_successes / self.num_pulls + np.sqrt(2 * np.log(i+1) / (self.num_pulls + 1e-9))
            chosen_arm = np.argmax(ucb_values)
            actual_arm = self.category_map[users_choices[i]]
            reward = 0 # could we work on getting 
            if (actual_arm == chosen_arm):
                reward+=1
            
            self.num_pulls[chosen_arm] += 1
            self.num_successes[chosen_arm] += reward
        
        ucb_values = self.num_successes / self.num_pulls + np.sqrt(2 * np.log(num_rounds+1) / (self.num_pulls + 1e-6))
        choose_next_arm = np.argmax(ucb_values)

        return self.categories[choose_next_arm]


class ThompsonSampling:
    def __init__(self, categories):
        self.num_arms = len(categories)
        self.categories = categories
        self.category_map = { categories[i]:i for i in range(len(categories)) }
        self.num_pulls = np.zeros(self.num_arms)
        self.num_successes = np.zeros(self.num_arms)
    
    def simulate(self, users_choices):
        num_rounds = len(users_choices)
        for i in range(num_rounds):
            sampled_probabilities = np.random.beta(self.num_successes + 1, self.num_pulls - self.num_successes + 1)
            chosen_arm = np.argmax(sampled_probabilities)

            actual_arm = self.category_map[users_choices[i]]
            reward = 0 # could we work on getting 
            if (actual_arm == chosen_arm):
                reward+=1
            
            self.num_pulls[chosen_arm] += 1
            self.num_successes[chosen_arm] += reward

        sampled_probabilities = np.random.beta(self.num_successes + 1, self.num_pulls - self.num_successes + 1)
        choose_next_arm = np.argmax(sampled_probabilities)

        return self.categories[choose_next_arm]
    
    

In [27]:
categories = ["clothes", "electronics", "shoes", "socks", "phone", "diary", "cow", "goat", "math", "jeans"]
UCB = UCB(categories)
users_choices = []

for i in range(100):
    x = np.random.choice(categories, p=[0.05, 0.5, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.1])
    users_choices.append(x)


In [54]:
print(users_choices)
UCB.simulate(users_choices)

['jeans', 'cow', 'math', 'socks', 'electronics', 'jeans', 'goat', 'clothes', 'electronics', 'electronics', 'math', 'jeans', 'electronics', 'shoes', 'phone', 'phone', 'phone', 'diary', 'cow', 'electronics', 'electronics', 'math', 'diary', 'goat', 'goat', 'goat', 'electronics', 'diary', 'phone', 'electronics', 'electronics', 'electronics', 'diary', 'electronics', 'electronics', 'jeans', 'electronics', 'electronics', 'jeans', 'electronics', 'electronics', 'electronics', 'electronics', 'goat', 'electronics', 'electronics', 'goat', 'jeans', 'math', 'cow', 'electronics', 'socks', 'electronics', 'jeans', 'cow', 'goat', 'diary', 'socks', 'clothes', 'shoes', 'electronics', 'electronics', 'electronics', 'electronics', 'electronics', 'clothes', 'diary', 'jeans', 'electronics', 'electronics', 'socks', 'electronics', 'electronics', 'cow', 'jeans', 'math', 'socks', 'electronics', 'electronics', 'electronics', 'jeans', 'electronics', 'electronics', 'diary', 'clothes', 'shoes', 'clothes', 'electronics

'electronics'